In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%config InlineBackend.figure_format = "retina"

import numpy as np
import xarray as xr
import sys,os,glob
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

sys.path.insert(0,'../../../')

from matplotlib import rcParams
plt.style.use(['fivethirtyeight','../../../twind/wind-paper.mplstyle'])
rcParams["savefig.dpi"] = 200
rcParams["figure.dpi"] = 200

import warnings

warnings.simplefilter("ignore")

# Simulation Model Tables 

We construct model PDFs based on the results from the TIGRESS simulation suite presented in [Paper I](https://ui.adsabs.harvard.edu/abs/2020ApJ...900...61K/abstract). We summarize model parameters and some integrated outflow propertes here using table files made available at [zenodo](https://doi.org/10.5281/zenodo.3872048) or [github](https://github.com/changgoo/tigress-wind-figureset/tree/v1.0.1). Mainly, the results are phase separated (three large bins in temperature or $c_s$) but outflow velocity integrated ($v_{\rm out}>0$). 

You can download the original [notebook](changgoo/tigress-wind-figureset/blob/master/tables/Example_scripts.ipynb) to reproduce tables and figures in [Paper I](https://ui.adsabs.harvard.edu/abs/2020ApJ...900...61K/abstract).

## Download and Prepare Tables 

In [3]:
# Download Tables
import urllib.request
import os

repo_url='https://changgoo.github.io/tigress-wind-figureset'
tbl_files=['table-mean.ecsv','table-mean-err.ecsv']
if not os.path.isdir('tables/'): os.mkdir('tables/')
for f in tbl_files:
    if not os.path.isfile(f):
        urllib.request.urlretrieve('{}/tables/{}'.format(repo_url,f),'tables/'+f)

In [4]:
# Read Tables with astropy:

from astropy.table import QTable,Table
tmean=Table.read('tables/table-mean.ecsv')
terr=Table.read('tables/table-mean-err.ecsv')

In [5]:
# add additional time scales for Table 2 in Paper I
import astropy.constants as ac
import astropy.units as au
tmean['torb']=(2*np.pi/tmean['Omega_0'].quantity).to('Myr')
tmean['tosca']=(2*np.pi/np.sqrt(4*np.pi*ac.G*tmean['rho_tot'].quantity)).to('Myr')
tmean['toscn']=(2.0*np.pi*tmean['H'].quantity/tmean['sigma_eff'].quantity).to('Myr')


# set format for more compact display
for k in tmean.keys():
    if tmean[k].info.dtype == 'float64':
        tmean[k].info.format = '15.2g'
        if k in terr: terr[k].info.format = '15.2g'

## Table 1: Model Parameters

In [6]:
table1_varlist=['model','Sigma_gas0','Sigma_star','rho_dm','Omega_0','z_star','R_0']
for k in table1_varlist:
    if tmean[k].info.dtype == 'float64':
        tmean[k].info.format = '15.3g'

tbl1=tmean[(tmean['z']=='H') & (tmean['phase']=='whole') ][table1_varlist]

tbl1.pprint_all()

model    Sigma_gas0      Sigma_star        rho_dm         Omega_0          z_star           R_0      
       solMass / pc2   solMass / pc2   solMass / pc3    km / (kpc s)         pc             kpc      
----- --------------- --------------- --------------- --------------- --------------- ---------------
   R2             150             450            0.08             100             245               2
   R4              50             208           0.024            53.7             245               4
   R8              12              42          0.0064              28             245               8
  R16            2.49            1.71         0.00143            11.9             245              16
 LGR2             150             110           0.015              50             500               2
 LGR4              60              50           0.005              30             500               4
 LGR8              12              10          0.0016              15             

- **Sigma_gas0**: initial gas surface density, $\Sigma_\text{gas,0}$
- **Sigma_star**: stellar surface density, $\Sigma_{*}$
- **rho_dm**: midplane dark matter density, $\rho_\text{dm}$ 
- **Omega**: angular velocity of galactic rotation, $\Omega$ 
- **R_0**: galactocentric radius, $R_0$ 
- **z_star**: scale height of stellar disk, $z_*$ 

## Table 2: Time Scales 

In [7]:
table2_varlist=['model','torb','toscn','tosca','tdep40','surf','sfr40']
tbl2=tmean[(tmean['z']=='H') & (tmean['phase']=='whole') ][table2_varlist]

tbl2.pprint_all()

model       torb           toscn           tosca           tdep40           surf             sfr40       
            Myr             Myr             Myr             Myr        solMass / pc2  solMass / (kpc2 yr)
----- --------------- --------------- --------------- --------------- --------------- -------------------
   R2              61              32              23              66              74                 1.1
   R4         1.1e+02              51              38         2.4e+02              29                0.12
   R8         2.2e+02         1.2e+02              75         2.1e+03              11              0.0051
  R16         5.2e+02         4.6e+02         3.1e+02         3.1e+04             2.5               8e-05
 LGR2         1.2e+02              52              48         1.5e+02              74                0.49
 LGR4           2e+02              87              80         4.2e+02              38                0.09
 LGR8         4.1e+02         2.2e+02         

- **torb**: orbit time, $t_\text{orb}=2\pi/\Omega$
- **toscn**: vertical oscillation time derived from numerical measures, $t_\text{osc,n}=2\pi H/\sigma_{\rm z,eff}$
- **tosca**: vertical oscillation time derived from input parameters, $t_\text{osc,a}=2\pi/(4\pi G\rho_{\rm tot})^{1/2}$
- **tdep40**: gas depletion time with SFR surface density in 40 Myr, $t_\text{dep,40}=\Sigma_\text{gas}/\Sigma_\text{SFR,40}$
- **surf**: mean gas surface density, $\Sigma_\text{gas}$
- **sfr40**: mean SFR surface density from star particles young than 40 Myr, $\Sigma_\text{SFR,40}$

_mean and error are determined from bootstrap resampling with a sample size of 10 for time series over_ $0.5<t/t_{\rm orb}<1.5$

## Table 3-1: Fluxes

In [8]:
z0='H' # height can be ('H','2H','500','1000')
table3_varlist1=['model','phase','mass','mom','energy','metal','metal_sn']
tbl3=tmean[tmean['z']==z0][table3_varlist1]

tbl3.pprint_all()

model phase         mass                  mom                 energy            metal              metal_sn     
            solMass / (kpc2 yr) km solMass / (kpc2 s yr) erg / (kpc2 yr) solMass / (kpc2 yr) solMass / (kpc2 yr)
----- ----- ------------------- ------------------------ --------------- ------------------- -------------------
   R2  cool                0.74                       50         7.2e+46               0.029              0.0032
   R2   int               0.063                       10         2.8e+46              0.0026             0.00056
   R2   hot                0.13                  1.4e+02         2.8e+48              0.0096              0.0062
   R2 whole                0.94                    2e+02         2.9e+48               0.041                0.01
   R4  cool                0.26                       12           1e+46              0.0081             0.00042
   R4   int               0.014                      1.8         4.1e+45             0.00047    

- **mass**: mass flux, $\overline{\mathcal{F}}_M$
- **mom**: momentum flux, $\overline{\mathcal{F}}_p$
- **energy**: energy flux, $\overline{\mathcal{F}}_E$
- **metal**: metal flux, $\overline{\mathcal{F}}_Z$
- **metal_sn**: SN-origin metal flux, $\overline{\mathcal{F}}_Z^{SN}$ 

_mean and error are determined from bootstrap resampling with a sample size of 10 for time series over_ $0.5<t/t_{\rm orb}<1.5$

## Table 3-2: Loading Factors

In [9]:
z0='H' # height can be ('H','2H','500','1000')
table3_varlist2=['model','phase','mass_loading','mom_loading',
                 'energy_loading','metal_loading','metal_sn_loading',]
tbl3=tmean[tmean['z']==z0][table3_varlist2]

tbl3.pprint_all()

model phase   mass_loading    mom_loading    energy_loading  metal_loading  metal_sn_loading
                                                                                            
----- ----- --------------- --------------- --------------- --------------- ----------------
   R2  cool            0.68           0.035          0.0064             1.3             0.14
   R2   int           0.058          0.0071          0.0025            0.11            0.025
   R2   hot            0.12             0.1            0.24            0.42             0.27
   R2 whole            0.86            0.14            0.25             1.8             0.44
   R4  cool             2.2           0.075           0.008             3.2             0.17
   R4   int            0.12           0.012          0.0032            0.19            0.028
   R4   hot            0.22            0.12            0.17             0.5             0.23
   R4 whole             2.5             0.2            0.18           

- **mass_loading**: mass loading factor, $\eta_M$
- **mom_loading**: mom loading factor, $\eta_p$
- **energy_loading**: energy loading factor, $\eta_E$
- **metal_loading**: mass loading factor, $\eta_Z$
- **metal_sn_loading**: SN-origin metal loading factor, $\eta_Z^{SN}$

_mean and error are determined from bootstrap resampling with a sample size of 10 for time series over_ $0.5<t/t_{\rm orb}<1.5$

## Table 4: Velocities and Metals 

In [10]:
z0='H' # height can be ('H','2H','500','1000')

table4_varlist=['model','phase','vout_flux','vB','Z','enrichment','fmass_sn','fmetal_sn']
tbl4=tmean[tmean['z']==z0][table4_varlist]

tbl4.pprint_all()

model phase    vout_flux           vB              Z           enrichment       fmass_sn       fmetal_sn   
                 km / s          km / s                                                                    
----- ----- --------------- --------------- --------------- --------------- --------------- ---------------
   R2  cool              69           1e+02           0.039             1.1           0.026            0.14
   R2   int         1.4e+02         2.1e+02           0.042             1.2           0.044            0.21
   R2   hot         5.8e+02         1.4e+03           0.072             2.1            0.23            0.63
   R2 whole         1.6e+02         5.6e+02           0.044             1.3           0.059            0.27
   R4  cool              47              67           0.032             1.1           0.011           0.068
   R4   int         1.1e+02         1.6e+02           0.034             1.1           0.023            0.13
   R4   hot         3.8e+02 

- **vout_flux**: characteristic outflow velocity, $\overline{v}_\text{out}$
- **vB**: Bernoulli velocity, $\overline{v}_{\mathcal{B}}$
- **Z**: outflow metallicity, $\overline{Z}$
- **enrichment**: metal enrichment factor, $\zeta$
- **fmass_sn**: fraction of SN-origin mass flux, $f_M^{SN}$
- **fmetal_sn**: fraction of SN-origin metal flux, $f_Z^{SN}$

_mean and error are determined from bootstrap resampling with a sample size of 10 for time series over_ $0.5<t/t_{\rm orb}<1.5$